In [ ]:
!pip install -r /content/drive/MyDrive/Colab_Notebooks/DWDM_Project/requirements.txt

In [ ]:
!pip install tabula-py

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Data')
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Scripts/Data_loading')
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Scripts/Data_processing')
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Data/cpbp.csv')
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Data/modis.csv')



In [ ]:
import os
print(os.path.exists('/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Data/cpcb.csv'))


False


In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add your scripts directory
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Scripts')

from Data_loading import (
    load_cpcb_data,
    prepare_cpcb_dataframe,
    calculate_missing_aqi,
    load_modis_data,
    load_health_data
)
from Data_processing import process_comprehensive_analysis

# File paths
cpcb_file   = '/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Data/cpcb_final.csv'
modis_file  = '/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Data/modis.csv'
health_pdfs = [
    '/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Data/Health/SRS_STAT_2020.pdf',
    '/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Data/Health/SRS_STAT_2021.pdf'
]

# Results directory
results_dir = '/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Results'
os.makedirs(results_dir, exist_ok=True)

print("🚀 Starting Enhanced India AQI Analysis Pipeline (2015–2020)")

# Phase 1: Load & Prepare CPCB Data
print("📊 PHASE 1: DATA PREPARATION")
raw_df   = load_cpcb_data(cpcb_file)
clean_df = prepare_cpcb_dataframe(raw_df)
clean_df = calculate_missing_aqi(clean_df)
clean_df.to_csv(f"{results_dir}/cleaned_cpcb_data.csv", index=False)

# Phase 2: Comprehensive Analysis
print("🧮 PHASE 2: COMPREHENSIVE ANALYSIS")
analysis = process_comprehensive_analysis(clean_df)

# Save analysis outputs
analysis['daily'].to_csv(f"{results_dir}/daily_summary.csv", index=False)
analysis['city'].to_csv(f"{results_dir}/city_summary.csv", index=False)
analysis['yearly'].to_csv(f"{results_dir}/yearly_summary.csv", index=False)
analysis['seasonal'].to_csv(f"{results_dir}/seasonal_summary.csv", index=False)

# Phase 3: Load MODIS Satellite Data
print("🛰️ PHASE 3: MODIS DATA")
modis_df = load_modis_data(modis_file)
modis_df.to_csv(f"{results_dir}/modis_data.csv", index=False)

print("✅ Pipeline completed. Results saved to:", results_dir)


🚀 Starting Enhanced India AQI Analysis Pipeline (2015–2020)
📊 PHASE 1: DATA PREPARATION


/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Scripts/Data_loading.py:34: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date_parsed'] = pd.to_datetime(df['date'], infer_datetime_format=True, errors='coerce')


🧮 PHASE 2: COMPREHENSIVE ANALYSIS
🛰️ PHASE 3: MODIS DATA


🏥 PHASE 4: HEALTH STATISTICS


Sep 19, 2025 1:35:48 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 19, 2025 1:35:48 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 19, 2025 1:35:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 19, 2025 1:35:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 19, 2025 1:35:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 19, 2025 1:35:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 19, 2025 1:35:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 19, 2025 1:35:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 19, 2025 1:35:49 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 19, 2025 1:35:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 19, 2025 1:35:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 19, 2025 1:35:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 19, 2025 1:35:50 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

   Loaded and saved health statistics
✅ Pipeline completed. Results saved to: /content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Results


In [6]:
import pandas as pd

# Load dataset
file_path = "/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Data/Health/HMIS report of all States and Districts Across the Months for 2019-20.csv"
df = pd.read_csv(file_path, encoding="ISO-8859-1")

# Map HMIS parameter names → your required columns
parameter_map = {
    "Inpatient - Asthma, Chronic Obstructive Pulmonary Disease (COPD), Respiratory infections": "Asthma/COPD Cases",
    "Childhood Diseases - Asthma": "Asthma Cases",
    "Inpatient – COPD": "COPD Cases",
    "Children admitted with upper respiratory infections": "Respiratory Hospitalizations",
    "Number of Child Deaths (1 -5 years) due to Pneumonia": "Respiratory Mortality",
    "Number of Adolscent / Adult deaths due to Respiratory diseases including infections (other than TB)": "Respiratory Mortality",
    "Childhood Diseases - Pneumonia": "Other Respiratory Illness Cases"
}

# Keep only matching rows
respiratory_df = df[df["Parameters"].isin(parameter_map.keys())].copy()

# Replace names with your clean column categories
respiratory_df["Parameters"] = respiratory_df["Parameters"].map(parameter_map)

# Reshape into long format
month_cols = [c for c in df.columns if "-" in c]
id_cols = ["State", "Parameters"]
if "District" in df.columns:
    id_cols.insert(1, "District")

resp_long = respiratory_df.melt(
    id_vars=id_cols,
    value_vars=month_cols,
    var_name="Month",
    value_name="count"
)

# Clean month names
resp_long["Month"] = resp_long["Month"].str.split("-").str[0].str.strip()

# Add Year (2019 Apr–Dec, 2020 Jan–Mar)
month_to_year = {m:2019 for m in ["April","May","June","July","August","September","October","November","December"]}
month_to_year.update({m:2020 for m in ["January","February","March"]})
resp_long["Year"] = resp_long["Month"].map(month_to_year)

# Drop NaNs
resp_long = resp_long.dropna(subset=["count"])

# Save updated filtered file
output_path = "/content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Data/Health/updated_Respiratory_HMIS_2019_20.csv"
resp_long.to_csv(output_path, index=False)

print("✅ Clean respiratory dataset saved at:", output_path)
print(resp_long.head(20))


✅ Clean respiratory dataset saved at: /content/drive/MyDrive/Colab_Notebooks/DWDM_Project/Data/Health/updated_Respiratory_HMIS_2019_20.csv
                State                       Parameters  Month    count    Year
0       A & N Islands  Other Respiratory Illness Cases  April      0.0  2019.0
1       A & N Islands                     Asthma Cases  April      0.0  2019.0
2       A & N Islands     Respiratory Hospitalizations  April     34.0  2019.0
3       A & N Islands                Asthma/COPD Cases  April    116.0  2019.0
4       A & N Islands            Respiratory Mortality  April      0.0  2019.0
5       A & N Islands            Respiratory Mortality  April      0.0  2019.0
6      Andhra Pradesh  Other Respiratory Illness Cases  April   1030.0  2019.0
7      Andhra Pradesh                     Asthma Cases  April    723.0  2019.0
8      Andhra Pradesh     Respiratory Hospitalizations  April   1112.0  2019.0
9      Andhra Pradesh                Asthma/COPD Cases  April  10915.0 

/tmp/ipython-input-353984913.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding="ISO-8859-1")
